In [2]:
import numpy as np
from numba import jit,cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from matplotlib import pyplot as plt
import math,time
import scipy.ndimage.filters
from threading import Thread,Lock
from ipywidgets import Image, HBox
import ipywidgets as widgets
from ipycanvas import Canvas
import io




class Cleanup():
    def __init__(self):
        self.threads = []
    def add(self,*threads):
        for thread in threads:
            self.threads.append(thread)
    def reset(self):
        for t in self.threads:
            t.isalive=False
            t.join()
        self.threads=[]
    def hard_reset(self):
        for a in self.threads:
            try:
                a.start()
            except:
                pass
        self.reset()
cleaner = Cleanup()

In [4]:
@cuda.jit
def fast_ising(grid,JB,mew,parity,rng_states,size):
    """Updates 1/4 of the grid by stochastically choosing points
    on a checkerboard grid and updating with boltzmann probability"""
    #find grid location
    i,j=cuda.grid(2)
    #get your first random number
    n = xoroshiro128p_uniform_float32(rng_states, i*size+j)
    #parity is 0 or 1, and tells you which half of the checkerboard to update
    if ((i+j)%2==parity and n < 0.5 and i<size and j<size):
        #sum the values of the nearest neighbors
        sum_=grid[i-1][j]+grid[i+1-size][j]+grid[i][j-1]+grid[i][j+1-size]
        #its grid*2-1 because I am using zero for spin down and 1 for spin up
        delta = 2.0 * (grid[i][j]*2-1)*(2*sum_-4+mew)
        #get our second random number
        n = xoroshiro128p_uniform_float32(rng_states, i*size+j)
        #update the cell according to the boltzmann distribution
        if (delta < 0 or 1-n < math.exp(-JB*delta)):
            grid[i][j]=1-grid[i][j]

In [8]:
@cuda.jit
def upscale(inarr,outarr):
    i,j=cuda.grid(2)
    iscale=outarr.shape[0]//inarr.shape[0]
    jscale=outarr.shape[1]//inarr.shape[1]
    outarr[i][j]=inarr[i//iscale][j//jscale]
class Render(Thread):
    def __init__(self, globalmem, canvas,dim=[512,512]):
        self.upscaled=np.zeros(dim)
        self.threads=(16,16)
        self.blocks=(int(np.ceil(dim[0] / 16)),int(np.ceil(dim[1] / 16)))
        self.grid_global_mem = globalmem
        self.canvas = canvas
        self.isalive=True
        super(Render, self).__init__()
    def run(self):
        while self.isalive:
            upscale[self.blocks,self.threads](self.grid_global_mem,self.upscaled)
            gridf=self.upscaled
            blue_channel = (1-gridf)*255
            red_channel = gridf*255
            green_channel = np.zeros_like(gridf)
            image_data = np.stack((red_channel, green_channel, blue_channel), axis=2)
            self.canvas.put_image_data(image_data, 0, 0)
            time.sleep(0.01)
            
            
class Ising(Thread):
    def __init__(self, N):
        self.N=N
        self.grid=np.asarray(np.random.random([N,N])>0.5,dtype=np.float32)
        self.threadsperblock = (16, 16)#should end up a multiple of 32 I think
        blockspergrid_x = int(np.ceil(self.grid.shape[0] / self.threadsperblock[0]))
        blockspergrid_y = int(np.ceil(self.grid.shape[1] / self.threadsperblock[1]))
        self.blockspergrid = (blockspergrid_x, blockspergrid_y)
        self.isalive=True
        self.rng_states = create_xoroshiro128p_states(self.grid.size, seed=1)
        self.grid_global_mem = cuda.to_device(self.grid)
        self.JB=5
        self.mew=0
        self.rspeed=100
        super(Ising, self).__init__()
    def run(self):
        while self.isalive:
            #parity=np.random.random()>0.5
            for x in range(self.rspeed):
                fast_ising[self.blockspergrid, self.threadsperblock](self.grid_global_mem,self.JB,self.mew,0,self.rng_states,self.N)
                fast_ising[self.blockspergrid, self.threadsperblock](self.grid_global_mem,self.JB,self.mew,1,self.rng_states,self.N)
            if self.rspeed<100:
                time.sleep(0.05)
                
                
model = Ising(256)
print(model.blockspergrid)
print(model.threadsperblock)
canvas = Canvas(width=512, height=512)
rend = Render(model.grid_global_mem,canvas)
cleaner.add(model,rend)
model.start()
rend.start()


def func(Beta,mew):
    model.JB=Beta
    model.mew=mew
    
plswork = widgets.Layout(
    width='50%'
)
    
x = widgets.FloatSlider(min=0,max=2,value=0.4407,step=0.001,layout=plswork)
x.style.handle_color = 'lightblue'
y = widgets.FloatSlider(min=-2,max=2,step=0.001,layout=plswork)
widgets.interact(func,Beta=x,mew=y)

canvas

(16, 16)
(16, 16)


interactive(children=(FloatSlider(value=0.4407, description='Beta', layout=Layout(width='50%'), max=2.0, step=…

Canvas(height=512, width=512)

In [9]:
cleaner.reset()